In [5]:
from neo4j import GraphDatabase, basic_auth
from dotenv import load_dotenv
import os



In [7]:
load_dotenv(os.path.join(os.path.expanduser('~'), '.spoke_neo4j_config.env'))
username = os.environ.get('NEO4J_USER')
password = os.environ.get('NEO4J_PSW')
url = os.environ.get('NEO4J_URI')
database = os.environ.get('NEO4J_DB')
